In [ ]:

# Telegram端步驟 https://pixnashpython.pixnet.net/blog/post/32391757-%E3%80%90telegram-api%E3%80%91python
'''
步驟一：在Telegram搜尋欄輸入BotFather並打開

步驟二：輸入 /newbot

步驟三：輸入機器人顯示名稱

步驟四：輸入機器人可搜尋名稱

步驟五：取得TOKEN複製起來(PYTHON用)
'''
#########
# https://www.freecodecamp.org/chinese/news/how-to-create-a-telegram-bot-using-python/
!pip install pyTelegramBotAPI


# 自動獲取工作資料
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:



# 18 版 $$$$$$$$$$$$$$$$$$$$$$$$$$
# https://chatgpt.com/share/ebfd449c-14c5-4dcc-a669-40d8f16f30c1

# 自動獲取工作資料
# chatgpt monica
# https://chat.openai.com/share/57c5b600-4c6a-40d2-b140-dd4761d6e5cd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time  # 新增此行，用於控制請求間的等待時間


# TG
import os
import re
import telebot
from google.colab import userdata

























class JobScraper:
    def __init__(self, keyword):
        self.keyword = keyword
        self.chrome_options = Options()
        self.chrome_options.add_argument('--headless')
        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-dev-shm-usage')
        self.chrome_options.add_argument('--disable-gpu')


    def _自動獲取香港勞工處工作資料(self):
        driver = webdriver.Chrome(options=self.chrome_options)
        driver.get("https://www1.jobs.gov.hk/0/tc/jobseeker/jobsearch/joblist/")

        # 填寫關鍵字並點擊搜尋按鈕
        search_box = driver.find_element(By.ID, "simp_searchKeyword")
        search_box.send_keys(self.keyword)
        search_button = driver.find_element(By.ID, "btnSearch")
        search_button.click()

        顯總料數 = 0
        _每頁量 = 20
        all工料 = []

        while True:

            # 等待搜尋結果加載完成
            WebDriverWait(driver, 10).until(EC.url_changes("https://www1.jobs.gov.hk/0/tc/jobseeker/jobsearch/joblist/"))

            # 獲取搜尋結果的HTML
            page_source = driver.page_source

            # 解析HTML並提取工作資料
            soup = BeautifulSoup(page_source, "html.parser")
            content_div = soup.find("div", id="content-innerdiv")
            job_listings = content_div.find_all("tr", class_="bg-white")

            # 取得資料總數
            if 顯總料數 == 0:
                total_jobs = soup.find("div", class_="py-2 d-lg-none").strong.text.strip()
                print("總共資料數:", total_jobs)
                顯總料數=1

                總共資料數 = f'總共資料數: {total_jobs}\n *** 資料來源:香港勞工處 ***\n\n'
                
            for job in job_listings:
                職位 = job.find("span").find("span").text.strip()
                刊日 = job.find_next("div", class_="col mb-2 mb-lg-0").span.text.strip()
                薪酬 = job.find_all("div", class_="col mb-2 mb-lg-0")[1].span.text.strip()
                地點 = job.find_all("div", class_="col mb-2 mb-lg-0")[2].span.text.strip()
                經驗 = job.find_all("div", class_="col mb-2 mb-lg-0")[3].span.text.strip()
                程度 = job.find_all("div", class_="col mb-2 mb-lg-0")[4].span.text.strip()
                網址 = "https://www1.jobs.gov.hk" + job.find_next("a")["href"]

                工料 = f'{職位}\n{薪酬}\n{地點}\n{經驗}\n{程度}\n{網址}\n$$$$ $$$$'
                all工料.append(工料)

            # 點擊下一頁按鈕
            if int(total_jobs) > _每頁量:
                # 點擊下一頁按鈕
                next_page_button = driver.find_element(By.XPATH, '//*[@id="content-innerdiv"]/div[4]/div/a[3]')
                next_page_button.click()
                _每頁量 += 20
            else:

              all工料.append(總共資料數)
              break

        # 關閉瀏覽器
        driver.quit()

        return all工料
    # 自動獲取香港勞工處工作資料 \


    def _爬取CareerJetHK內容(self):
        driver = webdriver.Chrome(options=self.chrome_options)
        all工料 = []
        page_number = 1

        while True:
            # 設置當前頁面的URL
            url = f"https://www.careerjet.com.hk/%E6%8B%9B%E8%81%98?s={self.keyword}&l=%E9%A6%99%E6%B8%AF&p={page_number}"
            driver.get(url)

            # 等待搜尋結果加載完成
            try:
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="search-content"]/ul'))
                )
            except Exception as e:
                print(f"搜尋結果頁面加載失敗或無資料: {e}")
                break

            # 取得搜尋結果的HTML
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # 抓取指定區塊的內容
            job_listings = soup.select('#search-content > ul > li')

            # 如果當前頁沒有資料，則結束爬取
            if not job_listings:
                print("已無更多資料，結束爬取。")
                break

            for job in job_listings:
                try:
                    # 內文網址
                    content_url = job.find('h2').find('a')['href']
                    內文網址 = "https://www.careerjet.com.hk" + content_url
                    # 工作標題
                    工作標題 = job.find('h2').find('a').text.strip()
                    # 公司名稱
                    公司名稱 = job.find('p').text.strip()
                    # 公司地址
                    公司地址 = job.find_all('ul')[1].find('li').text.strip()
                    # 簡介
                    簡介 = job.find('div').text.strip()

                    # 調整排列順序
                    content_data = {
                        "工作標題": 工作標題,
                        "公司名稱": 公司名稱,
                        "公司地址": 公司地址,
                        "簡介": 簡介,
                        "內文網址": 內文網址
                    }
                    工料 = f'{公司名稱}\n{工作標題}\n{公司地址}\n{簡介}\n{內文網址}\n$$$$ $$$$'

                    all工料.append(工料)
                except Exception as e:
                    print("錯誤:", e)
                    continue

            #print(f"第{page_number}頁資料已成功抓取。")
            page_number += 1  # 跳到下一頁

        # 關閉瀏覽器
        driver.quit()
        all工料.append(f'總共資料數: {len(all工料)}\n *** 資料來源:careerjet.com.hk ***\n\n')

        return all工料
    # 自動獲取careerjet.com.hk工作資料 \


    def 獲取所有工作資料(self):
        回1 = self._自動獲取香港勞工處工作資料()
        回2 = self._爬取CareerJetHK內容()
        return 回1 + 回2






























# Telegram bot配置
def _TG多工機器人():
    bot = telebot.TeleBot(userdata.get('tg2'))

    @bot.message_handler(commands=['start', 'hello', 'hi', 'k'])
    def send_welcome(message):
        bot.reply_to(message, "老闆您好!! 食左飯未?")


    @bot.message_handler(commands=['?'])
    def send_welcome(message):
        bot.reply_to(message, "\
        /$+工作名 = 搵工易(香港最新招聘資訊!!)\n\
        [資料來源:香港勞工處、careerjet.com.hk]\n\
        例: /$在家工作")


    @bot.message_handler(func=lambda message: re.search(r'\$', message.text))
    def handle_dollar_sign_commands(message):
        bot.reply_to(message, "正在獲取工作資料...請稍候...")
        keyword = message.text.replace("/$", "")

        # 使用 JobScraper 類別來獲取工作資料
        scraper = JobScraper(keyword)
        all_data = scraper.獲取所有工作資料()

        # 分割消息：將消息分割成更小的塊，每個塊的長度不超過Telegram API的限制（4096字節）
        MAX_MESSAGE_LENGTH = 4096
        chunks = []
        chunk = ''
        for data in all_data:
            if len(chunk) + len(data) + 1 < MAX_MESSAGE_LENGTH:
                chunk += data + '\n'
            else:
                chunks.append(chunk)
                chunk = data + '\n'
        if chunk:
            chunks.append(chunk)

        for chunk in chunks:
            try:
                bot.reply_to(message, chunk)
                time.sleep(1)  # 每發送一個訊息後等待 1 秒，避免觸發 API 限制
            except telebot.apihelper.ApiTelegramException as e:
                if e.result_json['error_code'] == 429:
                    retry_after = int(e.result_json['parameters']['retry_after'])
                    print(f"觸發 API 限制，需等待 {retry_after} 秒...")
                    time.sleep(retry_after)
                    bot.reply_to(message, chunk)
                else:
                    print(f"發送訊息時出現錯誤: {e}")


    print('$$$ TG多工機器人已執行 $$$')
    bot.infinity_polling()

# Telegram bot配置 \




















if __name__ == "__main__":
    _TG多工機器人()

# 18 版 $$$$$$$$$$$$$$$$$$$$$$$$$$